# Microproject - The Wright-Fisher Model of genetic drift


James Graydon | 10 February 2025  
Modified 17 February 2025

In [1]:
import random
import numpy as np

### Define a function to initialize the population

In [2]:
def init(N, f):
    """
    Return a list of zeroes and ones of length N 
    where the frequency of ones is f
    _____
    N: length of output list
    f: frequency of ones in output
    """
    n_1 = round(N * f)
    n_0 = N - n_1
    
    return ([0] * n_0) + ([1] * n_1) # creates a list of n_0 zeroes and n_1 ones

### Define a function to perform one WF generation

In [3]:
def step(pop):
    """ 
    Sample from the provided list with replacement
    _____
    pop: population
    """
    return random.choices(pop, k = len(pop)) 

### Define a function to bring it all together

In [4]:
def wf(N, f, ngens):
    """
    Call `init()` and `step()` to simulate genetic drift over time
    """   
    
    pop = init(N, f) # Initializes the population to be tested

    for _ in range(ngens):
        pop = step(pop) 

    freq_a = sum(pop) / N # Finds the frequency of the derived allele (a)

    print(f"generations: {ngens}; freq(a): {freq_a:0.2f}")

#### `wf()` example:

In [5]:
wf(N = 100, f = 0.2, ngens = 10)

generations: 10; freq(a): 0.32


### Challenge 1: Modified wf() function to test for fixation or loss of allele 1

In [6]:
def wf_fixloss(N, f, ngens):
    pop = init(N, f)

    gen = 0

    for _ in range(ngens):
        pop = step(pop)
        freq_a = sum(pop) / N # Finds the frequency of the derived allele (a)

        # Checks if the allele has become lost or fixed and breaks for-loop if so
        if freq_a == 0:
            print(f"The allele was lost after {gen} generations.")
            break 
        elif freq_a == 1:
            print(f"The allele became fixed after {gen} generations.")
            break 
        else:
            gen += 1 

    print(f"generations: {gen}; freq(a): {freq_a:0.2f}")

#### `wf_fixloss` example 1:

In [7]:
wf_fixloss(N = 100, f = 0.1, ngens = 1000)

The allele was lost after 55 generations.
generations: 55; freq(a): 0.00


#### `wf_fixloss` example 2:

In [8]:
wf_fixloss(N = 10, f = 0.9, ngens = 1000)

The allele became fixed after 2 generations.
generations: 2; freq(a): 1.00


### Challenge 2: Track allele frequency through time

In [9]:
def wf_track(N, f, ngens, return_n = None):
    pop = init(N, f) # Initializes the population to be tested
    l_freq = [f]

    for i in range(ngens):
        freq_a = sum(pop) / N # Finds the frequency of the derived allele (a)
        
        pop = step(pop) # Applies random changes to the `pop` consistent with genetic drift over time
        
        l_freq.append(freq_a)

    if return_n == None:
        l_freq = l_freq[0:ngens]
    else:
        l_freq = l_freq[0:return_n]
    
    return l_freq

wf_track(N = 10, f = 0.2, ngens = 1000, return_n = 10)

[0.2, 0.2, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

### Challenge 3: Wrap the wf() call inside an iterate_wf() function

In [10]:
def wf_fixloss_simple(N, f, gens):
    # A simplified version of `wf_fixloss()` which returns 1 if the allele becomes fixed or 0 if not
    pop = init(N, f)

    gen = 0

    for _ in range(gens):
        pop = step(pop)
        freq_a = sum(pop) / N # Finds the frequency of the derived allele (a)

        # Checks if the allele has become fixed, returns 1 if so
        if freq_a == 1:
            return 1 

    return 0

In [11]:
def iterate_wf(N, f, gens, times):
    # Runs `wf_fixloss_simple()` `times` times and returns the number of runs 
    # that resulted in the fixation of the dervied allele

    n_fixed = 0 # Initialize number of times wf function returns a fixed allele
    
    for _ in range(times):
        x = wf_fixloss_simple(N = N, f = f, gens = gens)
        if x == 1:
            n_fixed += 1

    freq_fixed = n_fixed / times

    return print(f"The dervied allele became fixed in ~{freq_fixed:.0%} of runs.")

iterate_wf(N = 100, f = 0.2, gens = 1000, times = 100)
    

The dervied allele became fixed in ~27% of runs.


*Disclaimer: ChatGPT-4o was used to simplify and verify the functionality the answers presented above.*

## Implemeting the Wright-Fisher Model as a Class

In [6]:
class Population:
    def __init__(self, N=10, f=0.2):
        self.N = N
        self.f = f
        self.freqs = []
    
        n_1 = round(N * f)
        n_0 = N - n_1
    
        self.pop = ([0] * n_0) + ([1] * n_1)

    def __repr__(self):
        return f"Population(N = {p.N}; f = {p.f})"

    def isMonomorphic(self):
        freq = sum(self.pop) / self.N
        if freq == 0 or freq == 1:
            return True
        else: 
            return False

    def step(self, ngens=1):
        for i in range(ngens):
            
            self.pop = random.choices(self.pop, k = self.N)

            freq = sum(self.pop) / self.N
            
            self.freqs.append(freq)
            
            if self.isMonomorphic():
                break 

p = Population()
p.step(ngens = 10)
p.freqs


[0.2, 0.3, 0.0]

In [ ]:
%%timeit

p = Population(1000000000)
p.N

In [ ]:
%%timeit

p = Population(1000000000)
len(p.pop)

## Numpy Challenge

In [184]:
class Population:
    def __init__(self, N=10, f=0.2, with_np=False):
        self.N = N
        self.f = f
        self.with_np = with_np

        n_1 = round(N * f)
        n_0 = N - n_1

        if with_np == True:
            self.pop = np.repeat([0, 1], [n_0, n_1])
        else:
            self.pop = ([0] * n_0) + ([1] * n_1)

    def step(self, ngens=1):
        for i in range(ngens):
            
            if self.with_np == True:
                #self.pop = np.random.choice(self.pop, size = self.N)
                self.pop[:] = self.pop[np.random.randint(0, high = self.N, size = self.N)]
            else:
                self.pop = random.choices(self.pop, k = self.N)

        return self.pop

In [235]:
N_size = 100000
n_gens = 100000

pop = Population(N = N_size)
pop_np = Population(N = N_size, with_np = True)


In [236]:
%%timeit
pop.step(ngens = n_gens)

6min 32s ± 20.5 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [237]:
%%timeit
pop_np.step(ngens = n_gens)

1min 13s ± 453 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Results and Interpretation

The following table summarizes the results from testing combinations of N = (100, 1000, 10000) and ngens = (100, 1000, 10000). All values are mean times.
| Population size (N) | Generations (ngens) | Time `with_np == False` | Time `with_np == True` |
|---------------------|---------------------|-------------------------|------------------------|
| `N = 100`           | `ngens = 100`       | 334 μs                  | 353 μs                 |
| `N = 100`           | `ngens = 1000`      | 3.23 ms                 | 3.49 ms                |
| `N = 100`           | `ngens = 10000`     | 32.5 ms                 | 35.8 ms                |
| `N = 1000`          | `ngens = 100`       | 3.89 ms                 | 617 μs                 |
| `N = 1000`          | `ngens = 1000`      | 36.4 ms                 | 6.08 ms                |
| `N = 1000`          | `ngens = 10000`     | 385 ms                  | 62.6 ms                |
| `N = 10000`         | `ngens = 100`       | 33.9 ms                 | 10.6 ms                |
| `N = 10000`         | `ngens = 1000`      | 367 ms                  | 112 ms                 |
| `N = 10000`         | `ngens = 10000`     | 3.75 s                  | 1.14 s                 |
| `N = 100000`        | `ngens = 100000`    | 6min 32s                | 1min 13s               |

As seen in the table, there is a **clear association between the size of the population/number of generations and the difference in performance speed between the array-based version of `Population` and the list-based version**. With smaller populations or fewer generations, the difference between the versions is minimal; in fact, the list-based version performs better on very small populations/generations, likely reflecting the lack of computational overhead needed to perform list-based calculations. However, as the population/generation sizes grow, the array-based version becomes substantially faster, generally achieving 3x the speed of the list-based version in the ranges tested, and almost 6x for N = 100000/ngens = 100000.

**In conclusion**, for very small datasets, a list-based approach is likely to be faster than an array-based approach. However, the advantage of an array-based approach quickly becomes apparent with larger datasets. Additionally, since the advantage for list-based methods for small datasets is very minor, it is likely more efficacious to use an array-based approach in any project that may become complex over time.

*Disclaimer: ChatGPT-4o was used to simplify and verify the functionality the answers presented above.*